In [6]:
# Installing the Required Libraries for the Notebook

import os

os.system("python -m pip install --upgrade pip -q")
os.system("pip install /kfs_private/movie_genre/katonic-1.0-py3-none-any.whl neattext tmdbsimple -q")
os.system("pip install scikit-learn==0.24.2 -q")

import tmdbsimple as tmdb
import time
import urllib
import json
import numpy as np
import pandas as pd
from joblib import load

import neattext as nt
import neattext.functions as nfx

from katonic.ml.client import MLClient

In [2]:
# remove some punctuation

def remove_punctuation(input_string):
    input_string = input_string.replace(',','')
    cleaned_string = input_string.replace('.','')    
    return cleaned_string

In [ ]:
os.environ["TMDB_API_KEY"] = "923a3621a665ddd778698f7e578a0490"
api_key = os.environ["TMDB_API_KEY"]

In [2]:
location = 's3://models/2/69a282d78a194fc19e08d14fad354f0f/artifacts/movie_genre_testcase_2_decision_tree_classifier'

In [3]:
# Syncing tmdb.API_KEY with Our Own api_key & Initiating a search 

tmdb.API_KEY = api_key
# Extracting the Genre IDs and Genre Names that are available in TMDB Database.

Genre_dict = tmdb.Genres().movie_list()["genres"]

# Mapping Genre IDs with Genre Names and making them a Dictionary.

Genres_id_name_dict = dict(zip([ movie["id"] for movie in Genre_dict], [ movie["name"] for movie in Genre_dict]))

# Let's separate out the IDs only.

genre_ids = list(Genres_id_name_dict.keys())

# By using the Above Ids we'll scrape the data from themoviedb.org for the base year of 2017.

movies = []
baseyear = 2022
 
print('Starting pulling movies from TMDB from each genre. This will take a while, please wait...')
done_ids=[]
for g_id in genre_ids:
    baseyear -= 1
    for page in range(1,6,1): # (1,6,1)
        time.sleep(1)
    
        url = 'https://api.themoviedb.org/3/discover/movie?api_key=' + api_key
        url += '&language=en-US&sort_by=popularity.desc&year=' + str(baseyear) 
        url += '&with_genres=' + str(g_id) + '&page=' + str(page)
 
        data = urllib.request.urlopen(url).read()
 
        dataDict = json.loads(data)
        movies.extend(dataDict["results"])
    last_movies = list(map(lambda x: x['title'],movies[-3:]))
    for title in last_movies:
        print('\t\t'+title)
    done_ids.append(str(g_id))
print("\tPulled movies for genres - "+','.join(done_ids))
print('\n')
 
# Remove duplicates movies if any Present in the Above movies.

movie_ids = [m['id'] for m in movies]
print ("Originally we had ",len(movie_ids)," movies")
movie_ids=np.unique(movie_ids)
seen_before=[]
no_duplicate_movies=[]
for i in range(len(movies)):
    movie=movies[i]
    if movie["overview"]:
        movie_id=movie['id']
        if movie_id in seen_before:
            continue
            print ("Seen before")
        else:
            seen_before.append(movie_id)
        no_duplicate_movies.append(movie)
print ("After removing duplicates we have ",len(no_duplicate_movies), " movies")

# Picking the movies which must have associated an overview.

movie_titles = []
movies_with_overviews=[] 
for i in range(len(no_duplicate_movies)):
    movie=no_duplicate_movies[i]
    overview=movie['overview']
    
    if len(overview)==0:
        continue
    else:
        movies_with_overviews.append(movie)
        movie_titles.append(movie['original_title'])

print("Done! Created a dataset where each movie must have an associated overview.\n")

# Making a DataFrame with the above text content and Target columns.

content = [movie["overview"] for movie in movies_with_overviews]

final_data = pd.DataFrame(content,columns=["overview"])

Starting pulling movies from TMDB from each genre. This will take a while, please wait...
		The Dark Knight Rises
		Mortal Kombat Legends: Battle of the Realms
		American Badger
		Missing Link
		Checkered Ninja
		Spies in Disguise
		Antz
		The Adventures of Tintin
		Over the Hedge
		Coming to America
		The House with a Clock in Its Walls
		Mom and Dad
		They Call Me Jeeg
		Life of Crime
		Green Room
		Weiner
		Holy Hell
		Lo and Behold: Reveries of the Connected World
		We Are Your Friends
		The Judge
		In the Heart of the Sea
		The Adventurer: The Curse of the Midas Box
		Minuscule: Valley of the Lost Ants
		Muppets Most Wanted
		The Rescuers
		Snow White: A Tale of Terror
		The House of Magic
		Day of the Siege
		Lady Gaga: On the Edge
		The Cup
		Eraserhead
		Guilty of Romance
		Phenomena
		Genius Within: The Inner Life of Glenn Gould
		Beirut - Cheap Magic Inside
		Obituary: Live Xecution
		Pontypool
		A Shot in the Dark
		The Conversation
		27 Dresses
		The Duchess
		Look Who's Ta

In [5]:
# Loading all the Transformers.

count_vectorizer = load("/kfs_private/movie_genre/CountVectorizer.joblib")

tf_idf_transformer = load("/kfs_private/movie_genre/TFIDF_transformer.joblib")

In [6]:
# Loading the Best model for the Inference.

mlc = MLClient(exp_name="movie_genre_testcase")

model_ml = mlc.load_model(location)

In [7]:
model_ml

DecisionTreeClassifier()

In [8]:
final_data["overview"] = final_data["overview"].apply(lambda x: remove_punctuation(x)) 
# Removing Stopwords from Overviews.
final_data["overview"] = final_data["overview"].apply(nfx.remove_stopwords)

X=count_vectorizer.transform(final_data["overview"]).toarray()

X_tfidf = tf_idf_transformer.transform(X).toarray()

X_train = pd.DataFrame(X_tfidf)

In [30]:
predictions = model_ml.predict(X_tfidf)

In [32]:
pd.DataFrame(predictions, columns = mlb.classes_)

NameError: name 'mlb' is not defined

In [ ]:
predictionsnb=[]
for i in range(X_test_tfidf.shape[0]):
    pred_genres=[]
    movie_label_scores=predsnb[i]
    for j in range(len(movie_label_scores)):
        #print j
        if movie_label_scores[j]!=0:
            genre=Genre_ID_to_name[genre_list[j]]
            pred_genres.append(genre)
    predictionsnb.append(pred_genres)